In [47]:
import tensorflow as tf

In [48]:
import math
import numpy as np
import pandas as pd
import random
import statistics
from collections import Counter
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredLogarithmicError
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Accuracy, Precision, AUC, Recall

In [49]:
train1_data = pd.read_csv('X_train1_oh.csv')
train2_data = pd.read_csv('X_train2_oh.csv')

test_data = pd.read_csv('X_test_oh.csv')


In [50]:
train1_data.shape

(140250, 385)

In [51]:
train1_data.head(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,374,375,376,377,378,379,380,381,382,383
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [52]:
train1_data=train1_data.iloc[:, 1:]

In [53]:
train1_data.shape

(140250, 384)

In [54]:
train2_data=train2_data.iloc[:, 1:]

In [55]:
train2_data.shape

(140250, 384)

In [56]:
test_data=test_data.iloc[:, 1:]

In [57]:
test_data.shape

(70126, 384)

In [58]:
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()

x_train1_scaled = mm_scaler.fit_transform(train1_data)
x_train2_scaled = mm_scaler.fit_transform(train2_data)

x_test_scaled = mm_scaler.transform(test_data)

In [60]:
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    print(output_units)
    self.encoder = Sequential(
        [
          Dense(512, activation="relu"),
          #Reducing the dimensionality of features from 218 to 32
          Dense(32, activation="relu")
        ]
    )

    self.decoder = Sequential(
        [
          Dense(256, activation="relu"),
          Dense(output_units, activation="sigmoid")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded
  
auto_encoder = AutoEncoders(len(x_test_scaled[0]))

auto_encoder.compile(
    loss='mean_squared_error',
    metrics=['mae', 'acc', tf.keras.metrics.Precision(), tf.keras.metrics.AUC(), tf.keras.metrics.Recall()],
    optimizer='adam'
)
callback = EarlyStopping(monitor='loss', patience=10)
history = auto_encoder.fit(
    x_train1_scaled, 
    x_train1_scaled, 
    epochs=100, 
    batch_size=2000, callbacks = [callback])
    #validation_data=(x_test_scaled, x_test_scaled)


384
Epoch 1/100
71/71 [==============================] - 7s 83ms/step - loss: 0.0549 - mae: 0.1125 - acc: 0.0027 - precision_9: 0.6691 - auc_9: 0.9139 - recall_7: 0.7148
Epoch 2/100
71/71 [==============================] - 6s 83ms/step - loss: 0.0074 - mae: 0.0125 - acc: 0.0000e+00 - precision_9: 0.9906 - auc_9: 0.9713 - recall_7: 0.9327
Epoch 3/100
71/71 [==============================] - 6s 83ms/step - loss: 0.0043 - mae: 0.0060 - acc: 0.0000e+00 - precision_9: 0.9963 - auc_9: 0.9792 - recall_7: 0.9564
Epoch 4/100
71/71 [==============================] - 6s 83ms/step - loss: 0.0038 - mae: 0.0049 - acc: 0.0000e+00 - precision_9: 0.9973 - auc_9: 0.9812 - recall_7: 0.9626
Epoch 5/100
71/71 [==============================] - 6s 83ms/step - loss: 0.0036 - mae: 0.0044 - acc: 0.0000e+00 - precision_9: 0.9976 - auc_9: 0.9819 - recall_7: 0.9635
Epoch 6/100
71/71 [==============================] - 6s 83ms/step - loss: 0.0034 - mae: 0.0041 - acc: 0.0000e+00 - precision_9: 0.9976 - auc_9: 0.9829

In [61]:
x_train1_scaled[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1.

In [62]:
auto_encoder.summary()


Model: "auto_encoders_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_18 (Sequential)  (None, 32)                213536    
                                                                 
 sequential_19 (Sequential)  (None, 384)               107136    
                                                                 
Total params: 320,672
Trainable params: 320,672
Non-trainable params: 0
_________________________________________________________________


In [63]:
f2=auto_encoder.predict(x_train2_scaled)
f3=auto_encoder.predict(x_test_scaled)


In [65]:
encoder_layer = auto_encoder.get_layer('sequential_18')
f2 = pd.DataFrame(encoder_layer.predict(x_train2_scaled))
f2 = f2.add_prefix('feature_')

In [67]:
#TODO Check if it is sequential_1
encoder_layer = auto_encoder.get_layer('sequential_18')
f3 = pd.DataFrame(encoder_layer.predict(x_test_scaled))
f3 = f3.add_prefix('feature_')

In [68]:
len(f2)

140250

In [69]:
f2

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
1,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
2,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
3,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
4,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140245,17.856840,15.496866,2.576589,19.360275,12.743767,0.0,0.0,0.0,18.674244,7.055664,...,0.000000,14.296276,0.730497,6.511075,2.561831,0.0,0.0,0.0,14.537747,10.688413
140246,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
140247,6.891916,5.400099,0.000000,0.576431,1.820116,0.0,0.0,0.0,12.939039,23.980650,...,1.619908,26.228575,11.130351,15.493748,24.002911,0.0,0.0,0.0,12.213769,0.000000
140248,22.851171,1.122389,0.000000,3.479761,3.944412,0.0,0.0,0.0,18.416704,0.000000,...,7.378504,22.320620,15.133915,14.909060,22.834076,0.0,0.0,0.0,9.450542,0.000000


In [70]:
len(f3)

70126

In [71]:
f3

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
1,17.856840,15.496866,2.576589,19.360275,12.743767,0.0,0.0,0.0,18.674244,7.055664,...,0.000000,14.296276,0.730497,6.511075,2.561831,0.0,0.0,0.0,14.537747,10.688413
2,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,0.000000,12.707335,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957
3,0.000000,8.305645,19.142534,11.442924,17.837807,0.0,0.0,0.0,12.570262,9.562721,...,0.000000,9.040876,15.615686,15.489821,5.483577,0.0,0.0,0.0,11.450768,11.722425
4,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70121,17.856840,15.496866,2.576589,19.360275,12.743767,0.0,0.0,0.0,18.674244,7.055664,...,0.000000,14.296276,0.730497,6.511075,2.561831,0.0,0.0,0.0,14.537747,10.688413
70122,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
70123,1.699281,7.902645,3.450099,0.000000,1.460008,0.0,0.0,0.0,4.788134,21.782103,...,25.940735,26.263226,9.377931,10.168983,14.014238,0.0,0.0,0.0,1.533820,0.000000
70124,0.000000,8.305645,19.142538,11.442926,17.837801,0.0,0.0,0.0,12.570261,9.562722,...,0.000000,9.040877,15.615685,15.489823,5.483575,0.0,0.0,0.0,11.450769,11.722427


In [72]:
from sklearn.ensemble import IsolationForest
if_model=IsolationForest(n_estimators=100, contamination=0.03)
if_model.fit(f2)
print(if_model)

score_pred = if_model.decision_function(f2)
#print(score_pred)
pred = if_model.predict(f3)
#print(pred)
print(type(pred))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


IsolationForest(contamination=0.03)
<class 'numpy.ndarray'>


In [73]:
arr = np.array([pred])
list1 = arr.tolist()
print(list1)
print(type(list1))

Counter(pred)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Counter({1: 68203, -1: 1923})

In [ ]:
#Second part is to train the positive samples to the same autoencoder architecture

In [74]:
f3['label'] = pred

In [75]:
f3.label

0        1
1        1
2        1
3        1
4        1
        ..
70121    1
70122    1
70123    1
70124    1
70125    1
Name: label, Length: 70126, dtype: int64

In [76]:
f3.shape

(70126, 33)

In [77]:
f3.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,1
1,17.856840,15.496866,2.576589,19.360275,12.743767,0.0,0.0,0.0,18.674244,7.055664,...,14.296276,0.730497,6.511075,2.561831,0.0,0.0,0.0,14.537747,10.688413,1
2,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,12.707335,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957,1
3,0.000000,8.305645,19.142534,11.442924,17.837807,0.0,0.0,0.0,12.570262,9.562721,...,9.040876,15.615686,15.489821,5.483577,0.0,0.0,0.0,11.450768,11.722425,1
4,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,1


In [78]:
#has only positive logs
p1 = f3[f3.label == 1]

In [79]:
p1.shape

(68203, 33)

In [80]:
p1.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,1
1,17.856840,15.496866,2.576589,19.360275,12.743767,0.0,0.0,0.0,18.674244,7.055664,...,14.296276,0.730497,6.511075,2.561831,0.0,0.0,0.0,14.537747,10.688413,1
2,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,12.707335,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957,1
3,0.000000,8.305645,19.142534,11.442924,17.837807,0.0,0.0,0.0,12.570262,9.562721,...,9.040876,15.615686,15.489821,5.483577,0.0,0.0,0.0,11.450768,11.722425,1
4,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,1


In [81]:
#has only negative logs
p2 = f3[f3.label == -1]

In [82]:
p2.shape

(1923, 33)

In [83]:
split_len = 0.7 * len(p1)
split_len

47742.1

In [84]:
auto2 = p1[:int(split_len)]
autotest = p1[int(split_len):]

In [85]:
#This is the training set and has only positive logs
auto2.shape

(47742, 33)

In [86]:
#To reset the index
trainp = auto2.reset_index(drop=True)

In [87]:
trainp.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,1
1,17.856840,15.496866,2.576589,19.360275,12.743767,0.0,0.0,0.0,18.674244,7.055664,...,14.296276,0.730497,6.511075,2.561831,0.0,0.0,0.0,14.537747,10.688413,1
2,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,12.707335,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957,1
3,0.000000,8.305645,19.142534,11.442924,17.837807,0.0,0.0,0.0,12.570262,9.562721,...,9.040876,15.615686,15.489821,5.483577,0.0,0.0,0.0,11.450768,11.722425,1
4,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,1


In [88]:
trainp = trainp.drop(['label'], axis=1)

In [89]:
trainp.shape

(47742, 32)

In [90]:
frames = [autotest, p2]

In [91]:
autoentest = pd.concat(frames)

In [92]:
testp = autoentest.reset_index(drop=True)

In [93]:
#Has some positive logs and some negative logs and is the testing set
testp.shape

(22384, 33)

In [94]:
testp.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,1
1,4.112792,15.657227,2.859466,4.664965,3.802977,0.0,0.0,0.0,4.151061,26.913092,...,24.066923,0.000000,1.093336,2.788831,0.0,0.0,0.0,2.928962,4.296680,1
2,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,12.707335,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957,1
3,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,1
4,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,1


In [95]:
testp = testp.drop(['label'], axis = 1)

In [96]:
testp.shape

(22384, 32)

In [97]:
len(trainp.iloc[0])

32

In [98]:
#This is the second autoencoder to train on the predicted positive and negative logs.
#This has the same architecture as the first autoencoder

In [104]:
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    print(output_units)
    self.encoder = Sequential(
        [
          Dense(512, activation="relu"),
          #Reducing the dimensionality of features from 218 to 32
          Dense(32, activation="relu")
        ]
    )

    self.decoder = Sequential(
        [
          Dense(256, activation="relu"),
          Dense(output_units, activation="sigmoid")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded
  
auto_encoder2 = AutoEncoders(len(trainp.iloc[0]))

auto_encoder2.compile(
    loss='mean_squared_error',
    metrics=['mae', 'acc', tf.keras.metrics.Precision(), tf.keras.metrics.AUC(), tf.keras.metrics.Recall()],
    optimizer='adam'
)

#Must pass the training set as both input and output 
#as the encoder will learn to compress the dataset 
history = auto_encoder2.fit(
    trainp, 
    trainp, 
    epochs=60, 
    batch_size=1000,
    #validation_data=(x_test_scaled, x_test_scaled)

)


32
Epoch 1/60
48/48 [==============================] - 2s 22ms/step - loss: 100.1443 - mae: 7.4596 - acc: 0.0212 - precision_13: 0.9206 - auc_13: 0.9378 - recall_11: 0.9635
Epoch 2/60
48/48 [==============================] - 1s 20ms/step - loss: 99.4716 - mae: 7.3990 - acc: 0.0265 - precision_13: 0.9583 - auc_13: 0.9506 - recall_11: 0.9997
Epoch 3/60
48/48 [==============================] - 1s 14ms/step - loss: 99.4715 - mae: 7.3988 - acc: 0.0265 - precision_13: 0.9583 - auc_13: 0.9506 - recall_11: 0.9997
Epoch 4/60
48/48 [==============================] - 1s 14ms/step - loss: 99.4715 - mae: 7.3988 - acc: 0.0265 - precision_13: 0.9583 - auc_13: 0.9506 - recall_11: 0.9997
Epoch 5/60
48/48 [==============================] - 1s 14ms/step - loss: 99.4715 - mae: 7.3987 - acc: 0.0265 - precision_13: 0.9583 - auc_13: 0.9506 - recall_11: 0.9997
Epoch 6/60
48/48 [==============================] - 1s 13ms/step - loss: 99.4715 - mae: 7.3987 - acc: 0.0265 - precision_13: 0.9583 - auc_13: 0.9506 - 

In [105]:
auto_encoder2.summary()

Model: "auto_encoders_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_30 (Sequential)  (None, 32)                33312     
                                                                 
 sequential_31 (Sequential)  (None, 32)                16672     
                                                                 
Total params: 49,984
Trainable params: 49,984
Non-trainable params: 0
_________________________________________________________________


In [107]:
encoder_layer = auto_encoder2.get_layer('sequential_30')
o1 = pd.DataFrame(encoder_layer.predict(trainp))
o1 = trainp.add_prefix('new_')

In [108]:
o1.shape

(47742, 32)

In [109]:
o1.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_22,new_feature_23,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31
0,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.0,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
1,17.856840,15.496866,2.576589,19.360275,12.743767,0.0,0.0,0.0,18.674244,7.055664,...,0.0,14.296276,0.730497,6.511075,2.561831,0.0,0.0,0.0,14.537747,10.688413
2,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,0.0,12.707335,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957
3,0.000000,8.305645,19.142534,11.442924,17.837807,0.0,0.0,0.0,12.570262,9.562721,...,0.0,9.040876,15.615686,15.489821,5.483577,0.0,0.0,0.0,11.450768,11.722425
4,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.0,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472


In [110]:
#Compute average and standard deviation of feature values of each sample
o1['o1_mean'] = o1.mean(axis = 1) 
o1['std'] = o1.std(axis = 1)
o1_std = o1['std'] = o1.std(axis = 1)

In [111]:
o1.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31,o1_mean,std
0,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,8.085794,6.828431
1,17.856840,15.496866,2.576589,19.360275,12.743767,0.0,0.0,0.0,18.674244,7.055664,...,0.730497,6.511075,2.561831,0.0,0.0,0.0,14.537747,10.688413,8.223319,7.334426
2,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957,8.749153,6.631445
3,0.000000,8.305645,19.142534,11.442924,17.837807,0.0,0.0,0.0,12.570262,9.562721,...,15.615686,15.489821,5.483577,0.0,0.0,0.0,11.450768,11.722425,8.213535,6.543677
4,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,8.085794,6.828431


In [117]:
#Calculating the threshold is done by multiplying std with a constant c.
#The c is chosen based on the predicted percentage of negative logs in the prediction using IF
threshold_value = o1_std * 0.02

In [118]:
print(threshold_value)

0        0.136569
1        0.146689
2        0.132629
3        0.130874
4        0.136569
           ...   
47737    0.130874
47738    0.136569
47739    0.132629
47740    0.136569
47741    0.130874
Length: 47742, dtype: float32


In [119]:
encoder_layer = auto_encoder2.get_layer('sequential_31')
o2 = pd.DataFrame(encoder_layer.predict(testp))
o2 = testp.add_prefix('new_')

In [120]:
o2.shape

(22384, 32)

In [121]:
o2.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_22,new_feature_23,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31
0,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
1,4.112792,15.657227,2.859466,4.664965,3.802977,0.0,0.0,0.0,4.151061,26.913092,...,26.996824,24.066923,0.000000,1.093336,2.788831,0.0,0.0,0.0,2.928962,4.296680
2,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,0.000000,12.707335,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957
3,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472
4,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,0.000000,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472


In [122]:
o2['o2_mean'] = o2.mean(axis = 1) 

In [123]:
o2.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_23,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31,o2_mean
0,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,8.085794
1,4.112792,15.657227,2.859466,4.664965,3.802977,0.0,0.0,0.0,4.151061,26.913092,...,24.066923,0.000000,1.093336,2.788831,0.0,0.0,0.0,2.928962,4.296680,6.202708
2,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,12.707335,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957,8.749153
3,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,8.085794
4,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,12.023054,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,8.085794


In [125]:
#If the mean of the o2 dataset is less than the threshold then that row is anamoly
o2.loc[o2['o2_mean'] < 0.146689, 'label'] = 'Anamoly'  
o2.loc[o2['o2_mean'] > 0.146689, 'label'] = 'not_Anamoly'  
 

In [126]:
o2.head(10)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31,o2_mean,label
0,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,8.085794,not_Anamoly
1,4.112792,15.657227,2.859466,4.664965,3.802977,0.0,0.0,0.0,4.151061,26.913092,...,0.000000,1.093336,2.788831,0.0,0.0,0.0,2.928962,4.296680,6.202708,not_Anamoly
2,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957,8.749153,not_Anamoly
3,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,8.085794,not_Anamoly
4,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,8.085794,not_Anamoly
5,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957,8.749153,not_Anamoly
6,0.000000,8.305645,19.142534,11.442924,17.837807,0.0,0.0,0.0,12.570262,9.562721,...,15.615686,15.489821,5.483577,0.0,0.0,0.0,11.450768,11.722425,8.213535,not_Anamoly
7,15.161313,6.722570,6.881887,8.900483,12.867071,0.0,0.0,0.0,16.173523,1.395920,...,19.867109,14.586211,5.066752,0.0,0.0,0.0,20.250465,8.786472,8.085794,not_Anamoly
8,19.511656,2.209018,1.295850,3.629351,4.506876,0.0,0.0,0.0,17.946297,1.821142,...,14.673095,15.419597,22.099266,0.0,0.0,0.0,8.401140,0.182935,6.943051,not_Anamoly
9,4.237618,15.232883,13.614224,20.169367,19.262150,0.0,0.0,0.0,16.557228,9.684449,...,6.822045,9.180683,0.000000,0.0,0.0,0.0,13.807485,14.107957,8.749153,not_Anamoly
